<img src='../img/EU-Copernicus-EUM-WEKEO_banner_logo.png' align='right' width='75%'></img>

<a href="../Index.ipynb"><< Index</a>
<br>
<a href="./1_1_OLCI_data_access.ipynb"><< OLCI data access</a>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href="./1_3_OLCI_coverage.ipynb">OLCI coverage >></a>

Copernicus Marine Training Service <br>
**Copyright:** 2022 EUMETSAT <br>
**License:** MIT

<html>
  <div style="width:100%">
    <div style="float:left"><a href="https://trainhub.eumetsat.int/ocean/sensors/1_2_OLCI_file_structure.ipynb"><img src="https://img.shields.io/badge/Launch-TrainHub-Blue.svg" alt="Open in TrainHub"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
    <div style="float:left"><a href="https://mybinder.org/v2/gl/eumetlab%2Focean%2Fsensors%2Flearn-olci/main?labpath=1_OLCI_introductory/1_2_OLCI_file_structure.ipynb"><img src="https://mybinder.org/badge_logo.svg" alt="Open in Binder"></a></div>
    <div style="float:left"><p>&emsp;</p></div>
    <div style="float:left"><a href="https://colab.research.google.com/github/eumetlab/ocean/sensors/learn-olci/blob/main/1_OLCI_introductory/1_2_OLCI_file_structure.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" /></a></div>
    <div style="float:left"><p>&emsp;</p></div>
    <div style="float:left"><a href="https://studiolab.sagemaker.aws/import/github/eumetlab/ocean/sensors/learn-olci/blob/main/1_OLCI_introductory/1_2_OLCI_file_structure.ipynb"><img src="https://studiolab.sagemaker.aws/studiolab.svg" alt="Open In Studio Lab"/></a></div>
    <div style="float:left"><p>&emsp;</p></div>
    <div style="float:left"><a href="https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/eumetlab/ocean/sensors/learn-olci&urlpath=1_OLCI_introductory/1_2_OLCI_file_structure.ipynb&branch=main"><img src="https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft" alt="Open in Planetary Computer"/></a></div>
  </div>
</html>

<div class="alert alert-block alert-success">
<h1>Learn OLCI: Introductory</h1></div>

<div class="alert alert-block alert-warning">
    
<b>PREREQUISITES </b>
    
The following modules are prerequisites for this notebook:
- **[1_1_OLCI_data_access.ipynb](1_1_OLCI_data_access.ipynb)** (this will download the required OLCI products for this notebook)

It is recommended to go through these modules before you start with this module.
</div>
<hr>

<div class="alert alert-info" role="alert">

## OLCI file structure
    
</div>

### Learning outcomes

At the end of this notebook you will know;
* SAFE format
* Seeing what components are inside a SAFE format file
* Seeing what variables are in each component

### Outline

<div class="alert alert-info" role="alert">

## <a id='TOC_TOP'></a>Contents

</div>
    
 1. [heading](#tag)

<hr>

In [1]:
# library imports
import xarray as xr
import glob
import os
from IPython.display import display, Markdown
import xml.etree.ElementTree as ET

In [2]:
# selecting SAFE directory
SAFE_directory = os.path.join(os.getcwd(), 'products', 
    'S3A_OL_1_EFR____20210717T101015_20210717T101315_20210718T145224_0179_074_122_1980_MAR_O_NT_002.SEN3')

In [3]:
# selecting SAFE manifest
SAFE_manifest = glob.glob(os.path.join(SAFE_directory, 'xfd*.xml'))[0]
display(Markdown('**Manifest file:** {}'.format(os.path.basename(SAFE_manifest))))

**Manifest file:** xfdumanifest.xml

In [4]:
# Reading SAFE manifest and outputting/collecting component names
tree = ET.parse(SAFE_manifest)
root = tree.getroot()
items = root.find('dataObjectSection')
SAFE_components = []
for item in items:
    display(Markdown("**File name:** {}".format(item[0][0].get('href'))))
    display(Markdown("*{}*".format(item[0][0].get('textInfo'))))
    print('')
    SAFE_components.append(os.path.join(SAFE_directory, item[0][0].get('href')))

**File name:** ./Oa01_radiance.nc

*TOA radiance for OLCI acquisition band Oa01*

**File name:** ./Oa02_radiance.nc

*TOA radiance for OLCI acquisition band Oa02*

**File name:** ./Oa03_radiance.nc

...

*Tie-Point Geometries Annotations*

**File name:** ./tie_meteo.nc

*Tie-Point Meteo Annotations*

**File name:** ./time_coordinates.nc

*Time Coordinates Annotations*

In [7]:
# Checking component structure and variables
for SAFE_component in sorted(SAFE_components):
    ds = xr.open_dataset(SAFE_component)
    display(Markdown('**Filename:** {}'.format(os.path.basename(SAFE_component))))
    display(ds)
    ds.close()

**Filename:** Oa01_radiance.nc

<xarray.Dataset>
Dimensions:        (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    Oa01_radiance  (rows, columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Radiance Oa01 Data Set

**Filename:** Oa02_radiance.nc

<xarray.Dataset>
Dimensions:        (rows: 4090, columns: 4865)
Dimensions without coordinates: rows, columns
Data variables:
    Oa02_radiance  (rows, columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Radiance Oa02 Data Set

**Filename:** Oa03_radiance.nc

...

<xarray.Dataset>
Dimensions:  (tie_rows: 4090, tie_columns: 77)
Dimensions without coordinates: tie_rows, tie_columns
Data variables:
    OAA      (tie_rows, tie_columns) float64 ...
    OZA      (tie_rows, tie_columns) float64 ...
    SAA      (tie_rows, tie_columns) float64 ...
    SZA      (tie_rows, tie_columns) float64 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Tie-Point Geometries Data Set

**Filename:** tie_meteo.nc

<xarray.Dataset>
Dimensions:                          (tie_rows: 4090, tie_columns: 77, tie_pressure_levels: 25, wind_vectors: 2)
Dimensions without coordinates: tie_rows, tie_columns, tie_pressure_levels, wind_vectors
Data variables:
    atmospheric_temperature_profile  (tie_rows, tie_columns, tie_pressure_levels) float32 ...
    horizontal_wind                  (tie_rows, tie_columns, wind_vectors) float32 ...
    humidity                         (tie_rows, tie_columns) float32 ...
    reference_pressure_level         (tie_pressure_levels) float32 ...
    sea_level_pressure               (tie_rows, tie_columns) float32 ...
    total_columnar_water_vapour      (tie_rows, tie_columns) float32 ...
    total_ozone                      (tie_rows, tie_columns) float32 ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Tie-Point Meteo Data Set

**Filename:** time_coordinates.nc

<xarray.Dataset>
Dimensions:     (rows: 4090)
Dimensions without coordinates: rows
Data variables:
    time_stamp  (rows) datetime64[ns] ...
Attributes: (12/16)
    absolute_orbit_number:  28200
    ac_subsampling_factor:  64
    al_subsampling_factor:  1
    comment:                 
    contact:                ops@eumetsat.int
    creation_time:          2021-07-18T14:52:24Z
    ...                     ...
    references:             S3IPF PDS 004.1 - i2r4 - Product Data Format Spec...
    resolution:             [ 270 294 ]
    source:                 IPF-OL-1-EO 06.11
    start_time:             2021-07-17T10:10:14.573099Z
    stop_time:              2021-07-17T10:13:14.494075Z
    title:                  OLCI Level 1b Product, Time Stamps Data Set

<hr>
<a href="https://gitlab.eumetsat.int/eumetlab/ocean">View on GitLab</a> | <a href="https://training.eumetsat.int/">EUMETSAT Training</a> | <a href=mailto:ops@eumetsat.int>Contact helpdesk for support </a> | <a href=mailto:Copernicus.training@eumetsat.int>Contact our training team to collaborate on and reuse this material</a></span></p>